In [68]:
# read json file

import json
import pandas as pd

with open('./Employee Review about their organization.json') as file:
    contents = file.read()
    contents = eval(contents)
    
df_list = []
for i in range(len(contents)):
    row_list = []
    row = contents[i]
    row_list.append(row["ReviewTitle"])
    row_list.append(row["CompleteReview"])
    row_list.append(row["URL"])
    row_list.append(row["Rating"])
    row_list.append(row["ReviewDetails"])
    df_list.append(row_list)
    
df = pd.DataFrame(data = df_list, columns = ['ReviewTitle', 'CompleteReview', 'URL', 'Rating', 'ReviewDetails'])

In [69]:
# 欄位處理

df['Company'] = df.URL.str.split('/')[:].str[4]
df['isCurrentEmployee'] = df.ReviewDetails.apply(lambda x: 0 if 'Former Employee' in x else 1)
rating = df.pop('Rating')
df['Rating'] = rating
df.drop(columns = ['URL', 'ReviewDetails'], inplace = True)
df.head()

,ReviewTitle,CompleteReview,Company,isCurrentEmployee,Rating
0,Productive,"Good company, cool workplace, work load little...",Reliance-Industries-Ltd,1,3.0
1,Stressful,1. Need to work on boss's whims and fancies 2....,Reliance-Industries-Ltd,0,3.0
2,Good Company for Every employee,"Good company for every Engineers dream, Full M...",Reliance-Industries-Ltd,0,5.0
3,Productive,I am just pass out bsc in chemistry Typical da...,Reliance-Industries-Ltd,1,5.0
4,Non productive,Not so fun at work just blame games Target pe...,Reliance-Industries-Ltd,0,1.0


In [70]:
# 測試用而已，不用理這個
print('Review：\n' + df.loc[1]['CompleteReview'], '\n')
print('Company：\n' + df.loc[1]['Company'], '\n')
print('Rating：\n' + df.loc[1]['Rating'], '\n')

Review：
1. Need to work on boss's whims and fancies 2. Priorities keep changing 3. No regards for work life balance 4. Completing the work before time is stressed too much than completing it well. Involves lots of reworking, blame games; etc. 5. No job boundaries, you will be asked to do any work depending on the requirements. 

Company：
Reliance-Industries-Ltd 

Rating：
3.0 



In [71]:
# 測試用而已，不用理這個
import numpy as np

def calculate_mean_score(company):
    scores = [float(value) for value in df[df['Company'] == company].Rating.values]
    return np.mean(scores)
print(calculate_mean_score('Google'))
print(calculate_mean_score('Kpmg'))

4.48581560283688
3.9604221635883907


In [72]:
# 測試用而已，不用理這個
set(df.Company)

{'Accenture',
 'Alstom',
 'Amazon.com',
 'American-Express',
 'Apollo-Hospitals',
 'Axis-Bank',
 'Barclays',
 'Bharti-Airtel-Limited',
 'Bosch',
 "Byju's",
 'Capgemini',
 'Citi',
 'Cognizant-Technology-Solutions',
 'Concentrix',
 'DHL',
 'Dell-Technologies',
 'Deloitte',
 'Deutsche-Bank',
 'Ericsson',
 'Ey',
 'Facebook',
 'Flipkart.com',
 'Google',
 'HP',
 'HSBC',
 'Hdfc-Bank',
 'Hinduja-Global-Solutions',
 'Honeywell',
 'Hyatt',
 'IBM',
 'Icici-Prudential-Life-Insurance',
 'Indeed',
 'Infosys',
 'JPMorgan-Chase',
 'Jio',
 'Jll',
 'Kotak-Mahindra-Bank',
 'Kpmg',
 'L&T-Technology-Services-Ltd.',
 'Maersk',
 'Mahindra-&-Mahindra-Ltd',
 'Marriott-International,-Inc.',
 'Microsoft',
 'Mphasis',
 'Muthoot-Finance',
 'Nokia',
 'Oracle',
 'Reliance-Industries-Ltd',
 'Ride.swiggy',
 'Standard-Chartered-Bank',
 'Sutherland',
 'Tata-Aia-Life',
 'Tata-Consultancy-Services-(tcs)',
 'Teleperformance',
 'UnitedHealth-Group',
 'Vodafoneziggo',
 'Wells-Fargo',
 'Wns-Global-Services',
 'Yes-Bank'}

In [73]:
# 有 14 多萬筆評論
len(df)

145209

### 我從這裡開始

In [74]:
df['Rating'] = df['Rating'].astype(float)
df['Rating'].mean()

4.053660585776364

In [75]:
mean = df['Rating'].mean()
df['Rating'].iloc[df['Rating'] < mean] = 0
df['Rating'].iloc[df['Rating'] >= mean] = 1

/usr/local/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [76]:
df['Rating']

0         0.0
1         0.0
2         1.0
3         1.0
4         0.0
         ... 
145204    0.0
145205    0.0
145206    0.0
145207    0.0
145208    1.0
Name: Rating, Length: 145209, dtype: float64

In [77]:
from sklearn.model_selection import train_test_split
X = df.iloc[:, :-1]
y = df.iloc[:, -1]
#先抽10%
X_train, X_evaluate, y_train, y_evaluate = train_test_split(X, y, test_size = 0.1, random_state = 8, stratify = y)

In [78]:
len(X_evaluate)

14521

In [79]:
#再9:1
X_train, X_test, y_train, y_test = train_test_split(X_evaluate, y_evaluate, test_size = 0.1, random_state = 8)

In [80]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state = 8)

In [81]:
print("X_train shape:", len(X_train))
print("y_train shape:", len(y_train))
print("X_val shape:", len(X_val))
print("y_val shape:", len(y_val))

X_train shape: 11761
y_train shape: 11761
X_val shape: 1307
y_val shape: 1307


In [82]:
X_train['id'] = X_train.index
X_train

,ReviewTitle,CompleteReview,Company,isCurrentEmployee,id
101323,pathatic company to work with,Minacs is completely a mallu and bangali manag...,Concentrix,0,101323
7624,Productive and learning workplace,This job requires me to coordinate and control...,Alstom,1,7624
23754,Good company with good cooperative management,Day to day work was to handle the employees da...,Mahindra-&-Mahindra-Ltd,0,23754
68128,Good working culture,"Good environment,supportive employees,ambiance...",American-Express,0,68128
39263,Good organization,"It depends on team to team, however, its worth...",Oracle,0,39263
...,...,...,...,...,...
89144,Good work place,Learn many technical things and overcome from ...,Tata-Consultancy-Services-(tcs),1,89144
74859,Highly competitive and challenging in a good way,Our team handles different things 24*7. Every ...,Amazon.com,0,74859
53276,good workplace,plenty of opportunities to learn\r \r good wor...,Bosch,0,53276
74832,Productive and fun work place,A typical day at work would always start with ...,Ericsson,0,74832


### 斷詞

In [83]:
#沿用 PA1 的內容
def syllables(word):
    #當字尾是e時音節數會減一
    if word.endswith("e"):
        word.replace("e", "")
    
    #當字裡包含a e i o u y時，會增加音節
    word = word.replace("a", ",")
    word = word.replace("e", ",")
    word = word.replace("i", ",")
    word = word.replace("o", ",")
    word = word.replace("u", ",")
    word = word.replace("y", ",")
    
    count = word.count(",")
    return count

In [84]:
#沿用 PA1 的內容
def tokenization(text):
    punctuation = [",", "'", ".", "?", "!", "(", ")", ":", "`", "``", "%", "{", "}", ";", "*", "#", "+", "$", 
                   "\\", "/", "_", "=", "^", "&", "<", ">", "@", "|", "[", "]"]
    for i in punctuation:
        text = text.replace(i, " ")
    
    #新增移除數字
    numbers = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0"]
    for j in numbers:
        text = text.replace(j, " ")
    
    text = text.replace("\n", " ")
    text = text.replace("\t", " ") 
    text = text.replace("\r", " ") 
    text = text.replace('"', "")
    
    #檢查 hyphens
    if "-" in text:
        t = text.count("-")
        
        #如果那個字的 hyphen 數小於2時，就會進入音節數的判斷
        if t < 2:
            count_syllables_word = text.split("-")
            s = syllables(count_syllables_word[0]) #計算第一個字音節
            
            #如果那個字的音節數小於2時，就會把字合併
            if(s < 2): 
                text = text.replace("-", "") #ex. co-worker --> coworker  
            else:
                text = text.replace("-", " ") #ex. semi-supervised --> semi supervised
        else:
            text = text.replace("-", " ") #ex. state-of-art --> state of art

    
    text = text.split(" ")
    text = list(filter(None, text)) #remove null
    
    return text

In [85]:
# #沿用 PA1 的內容
# stopword_list = []
# stopword = open("./stopwords.txt", 'r')
# for s in stopword.readlines():
#     s = s.replace("\n", "")
#     s = s.replace("'", " ") 
#     s = s.split(" ")

#     stopword_list.extend(s)

# stopword.close()

In [90]:
from nltk.stem.porter import *
import nltk
from nltk.corpus import stopwords
stemmer = PorterStemmer()
stopword_list = set(stopwords.words('english'))

docID = []
t_index = []
term = []
df = []
tf = []

for i in range(len(X_train)):
    token_list = []    

    token_list.extend(tokenization(X_train['CompleteReview'].iloc[i].encode('utf-8', 'ignore').decode('ascii', 'ignore')))
    
    #lower case
    for j in range (len(token_list)):
        if(isinstance(token_list[j], str)): #only string has to lower case 
            token_list[j] = token_list[j].lower()  
    
    #stemming
    new_token_list = [stemmer.stem(token) for token in token_list]    
    
    #stopword removal
    filtered_tokens = [token for token in new_token_list if token not in stopword_list] 
    
    #紀錄 doc_i 中的 docID, term, tf
    for t in filtered_tokens: 
        docID.append(X_train['id'].iloc[i])
        term.append(t)
        tf.append(filtered_tokens.count(t)) #tf 的平均是用原始檔案中的字數去除

In [91]:
#step_one: 在這個步驟中將 term 跟 docID 取出
step_one = []
for i in range(len(term)):
    tmp = [term[i], str(docID[i])] #[apple, 1]
    step_one.append(tmp)

In [92]:
#step_two: 這個步驟中將 term 按照字母順序排列，並去除重複的字及記錄 df

import numpy as np
#sort alphabetically
step_two = sorted(step_one, key=lambda l:l[0])

step_two = np.array(step_two) #list to numpy

step_two = np.unique(step_two, axis=0) #去除同一個 docID 中的相同字
step_two = step_two.T
t, df = np.unique(step_two[0], return_counts=True) #計算不同檔案中的相同字數量
print(t)
print(df)

['aa' 'aaa'
 'aaaaaaaahhhhhjbbbhhzhhshsbsbzbzbzbzbzbsbsbzbsbsbsbsbsbsbsbsbsbsbbbsbsbsbsbsbsbsnsnsbsbsbsbsjsbshsbsbjsjsjsjjajjajajajajsjsjsjsjjsjajsjsjsjsjsjjsjsjsjsjsjjsjsjajjsjjajh'
 ... 'zoom' 'zoveili'
 'zzcadasdgadsgadgdddaddddddgggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggg']
[2 1 1 ... 1 1 1]


In [93]:
my_dic ={}
my_dic["term"] = t
my_dic["df"] = df
my_dic["t_index"] = [i for i in range (1, len(t)+1)] #給定 term index (從1開始)

In [94]:
#把字典的字和對應到的文件 id 取出，方便之後找字對應到的 class
dict_ids = {}
c = 0
for t in step_two[0]:
    if t in term:
        if t not in dict_ids:
            dict_ids[t] = list()
        dict_ids[t].append(step_two[1,c])      
        
    c+=1

In [95]:
#寫入 dictionary.txt 中
dic = open("dictionary.txt", "w")
dic.write("t_index term df")
for i in range (len(my_dic["term"])):
    dic.write("\n" + str(my_dic["t_index"][i]).encode('utf-8','ignore').decode("utf-8") + " " + str(my_dic["term"][i]).encode('utf-8','ignore').decode("utf-8") + " " + str(my_dic["df"][i]).encode('utf-8','ignore').decode("utf-8"))
    
dic.close()

In [96]:
#將 docID, term, tf, t_index, df 放到同一個 dataframe，以方便計算 tf-idf
#dataframe for tf
import pandas as pd
tf_dataframe = pd.DataFrame(list(zip(docID, term, tf)), columns =['docID', 'term', 'tf'])

In [97]:
tf_dataframe

,docID,term,tf
0,101323,minac,1
1,101323,complet,1
2,101323,mallu,1
3,101323,bangali,1
4,101323,manag,3
...,...,...,...
263040,72228,time,1
263041,72228,share,1
263042,72228,experi,1
263043,72228,us,1


In [98]:
#dataframe for dictionary
dict_df = pd.read_csv('dictionary.txt', delimiter = " ", keep_default_na=False)

In [99]:
#join tf_dataframe with dataframes
df3 = tf_dataframe.join(dict_df.set_index('term'), on='term')

In [100]:
df3

,docID,term,tf,t_index,df
0,101323,minac,1,4996,6
1,101323,complet,1,1489,327
2,101323,mallu,1,4757,1
3,101323,bangali,1,656,1
4,101323,manag,3,4770,4035
...,...,...,...,...,...
263040,72228,time,1,7926,1632
263041,72228,share,1,7029,262
263042,72228,experi,1,2760,1638
263043,72228,us,1,8328,420


In [101]:
#先計算 tf-idf
N = len(X_train)
tfidf = df3['tf'] * np.log10(N / df3['df'])
df3 = df3.assign(tf_idf = tfidf)

In [102]:
#將 tf-idf 轉為 unit vector
new_df3 = df3.copy()
for i in (X_train['id']):
    #完整 vector space 大小
    matrix = df3.shape[0]   
    vector_space = np.zeros(matrix)
    rows = df3.loc[df3['docID'] == i]
    
    #取出對應的 docID 的 vector 後算距離
    vector_space[rows['t_index']] = rows['tf_idf']
    new_tfidf = rows.iloc[:, 5] / np.linalg.norm(vector_space)
    
    #改成 unit vector
    new_df3.loc[new_df3['docID'] == i, 'tf_idf'] = new_tfidf

In [103]:
new_df3

,docID,term,tf,t_index,df,tf_idf
0,101323,minac,1,4996,6,0.209035
1,101323,complet,1,1489,327,0.098787
2,101323,mallu,1,4757,1,0.258441
3,101323,bangali,1,656,1,0.258441
4,101323,manag,3,4770,4035,0.088496
...,...,...,...,...,...,...
263040,72228,time,1,7926,1632,0.125933
263041,72228,share,1,7029,262,0.242571
263042,72228,experi,1,2760,1638,0.125699
263043,72228,us,1,8328,420,0.212480


### For feature selection

In [104]:
#step_two: 這個步驟中將 term 按照字母順序排列，並去除重複的字及記錄 df

import numpy as np
#sort alphabetically
step_two = sorted(step_one, key=lambda l:l[0])

step_two = np.array(step_two) #list to numpy
step_two = np.unique(step_two, axis=0) #去除同一個 docID 中的相同字

step_two = step_two.T

t, df = np.unique(step_two[0], return_counts=True) #計算不同檔案中的相同字數量
df = df.tolist()

In [105]:
my_dic ={}
my_dic["term"] = t
my_dic["df"] = df
my_dic["t_index"] = [i for i in range (1, len(t)+1)] #給定 term index (從1開始)

In [106]:
tf_dataframe = pd.DataFrame(list(zip(my_dic["t_index"], my_dic["term"], my_dic["df"])), columns =['t_index', 'term', 'df'])
tf_dataframe

,t_index,term,df
0,1,aa,2
1,2,aaa,1
2,3,aaaaaaaahhhhhjbbbhhzhhshsbsbzbzbzbzbzbsbsbzbsb...,1
3,4,abandon,1
4,5,abap,7
...,...,...,...
8858,8859,zonathan,1
8859,8860,zone,18
8860,8861,zoom,1
8861,8862,zoveili,1


In [107]:
#把字典的字和對應到的文件 id 取出，方便之後找字對應到的 class
dict_ids = {}
c = 0
for t in step_two[0]:
    if t in term:
        if t not in dict_ids:
            dict_ids[t] = list()
        dict_ids[t].append(step_two[1,c])      
        
    c+=1

In [108]:
dict_id_df = pd.DataFrame([dict_ids])
dict_id_df = dict_id_df.T
dict_id_df

,0
aa,"[1998, 47146]"
aaa,[139002]
aaaaaaaahhhhhjbbbhhzhhshsbsbzbzbzbzbzbsbsbzbsbsbsbsbsbsbsbsbsbsbbbsbsbsbsbsbsbsnsnsbsbsbsbsjsbshsbsbjsjsjsjjajjajajajajsjsjsjsjjsjajsjsjsjsjsjjsjsjsjsjsjjsjsjajjsjjajh,[116587]
abandon,[84077]
abap,"[114268, 126675, 144548, 38136, 39758, 46013, ..."
...,...
zonathan,[30192]
zone,"[108988, 110507, 11283, 115295, 133605, 138697..."
zoom,[37921]
zoveili,[32477]


In [111]:
#把 id 加入大表格中檢查
tf_dataframe['ids'] = dict_id_df[0].values

In [112]:
tf_dataframe

,t_index,term,df,ids
0,1,aa,2,"[1998, 47146]"
1,2,aaa,1,[139002]
2,3,aaaaaaaahhhhhjbbbhhzhhshsbsbzbzbzbzbzbsbsbzbsb...,1,[116587]
3,4,abandon,1,[84077]
4,5,abap,7,"[114268, 126675, 144548, 38136, 39758, 46013, ..."
...,...,...,...,...
8858,8859,zonathan,1,[30192]
8859,8860,zone,18,"[108988, 110507, 11283, 115295, 133605, 138697..."
8860,8861,zoom,1,[37921]
8861,8862,zoveili,1,[32477]


In [113]:
#計算 train data 的數量
count_docs = len(X_train)

In [114]:
count_docs

11761

In [42]:
#index_list = list(X_train.index.values)

In [115]:
train_df = pd.DataFrame(y_train)
train_df['id'] = train_df.index
train_df = train_df.rename(columns = {'Rating':'class'})

In [116]:
train_df

,class,id
101323,1.0,101323
7624,0.0,7624
23754,1.0,23754
68128,1.0,68128
39263,0.0,39263
...,...,...
89144,0.0,89144
74859,1.0,74859
53276,1.0,53276
74832,1.0,74832


### 對 dictionary 做 feature selection (LLR)

In [ ]:
import math
final_llr = []
term = tf_dataframe['term'].tolist()
cl = [0, 1]
N = count_docs #總共幾篇文章

for t in term:
    llr = []
    #該字有被那些文件使用到
    get_id = dict_ids[t]

    for c in cl:
        n11, n01, n10, n00, on_topic = 0, 0, 0, 0, 0
        for i in get_id:            
            #取得文件對應的類別
            get_class = train_df.loc[train_df['id'] == int(i)]
            get_class = int(get_class['class'])
            
            #屬於該類別就 n11 加一
            if (get_class == c):
                n11 += 1
                
            #不屬於該類別的話 n01 加一
            else:
                n01 += 1
        
        #計算 class c 有幾篇
        on_topic = len(train_df['id'].loc[train_df['class'] == c].values)
 
        n10 = on_topic - n11
        n00 = N - n11 - n01 - n10

        pt = (n11 + n01) / N
        p1 = n11 / (n11 + n10)
        p2 = n01 / (n01 + n00)
        
        #公式計算
        a = pt**(n11)
        b = (1-pt)**(n10)
        c = pt**(n01)
        d = (1-pt)**(n00)
        e = p1**(n11)
        f = (1-p1)**(n10)
        g = p2**(n01)
        h = (1-p2)**(n00)
        
#         print(e)
#         print(f)
#         print(g)
#         print(h)
        #print(e*f*g*h+0.001)
        llr.append(-2*math.log((a*b*c*d+0.001) / (e*f*g*h+0.001))) #add 1e-5 for smoothing 

    llr_mean = (sum(llr)/len(llr)) #max or average?
    final_llr.append(llr_mean)

In [ ]:
#choose 前 500
term = np.array(term)
selected_term = term[np.argpartition(final_llr,-500)[-500:]]
print(selected_term)